In [1]:
!pip install llama_cloud_services openpyxl

In [2]:
import pydantic
print(pydantic.__version__)
from pydantic import BaseModel, Field, model_validator



2.11.5


In [ ]:
import os
import pandas as pd
from llama_cloud_services import LlamaExtract
from pydantic import BaseModel, Field, model_validator
from financial_schemas_incomestatement_final import generate_income_statement_schema 
#from dotenv import load_dotenv


In [ ]:
FISCAL_YEAR = 2024  #Change the year if you want different years
IncomeStatement_2024_25 = generate_income_statement_schema(FISCAL_YEAR)

PDF_ROOT = "private_universities/university_pdfs"  # Change this to the point to the directory where you are storing the pdfs after scraping
OUTPUT_ROOT = "output_incomestatement_final"  # Make this point to the directory/folder where you want to store the excel files with information extracted
os.makedirs(OUTPUT_ROOT, exist_ok=True)  
AGENT_ID = "43c63d27-55a6-4268-abb4-26d9ed82480f" #Different based on what agent you are using. Check llamacloud for the list of agents we have under the pro account
api_key = os.getenv("LLAMACLOUD_API_KEY")

In [ ]:
extractor = LlamaExtract(
    api_key="llx-63CU3PdyDo0d230ureocmy9JOHgnPwYgE2HETi55DqzYCIpy", # Add your Llamacloud API Key 
    project_id="8c10e62e-3810-4193-915d-d2d11105826d"   #Change the project ID only if Luis has asked you. This is dependent on the llamacloud account
)


#agent = extractor.create_agent(name = "endowment-parser-2024", data_schema=EndowmentAndInvestmentLevels_2024_25)

agent = extractor.get_agent(id = AGENT_ID)

#uncomment the following lines if you updated the schema
agent.data_schema = IncomeStatement_2024_25.schema()
agent.save()
agent = extractor.get_agent(id = AGENT_ID)


/var/folders/m1/8j3nd3m95y1brb0j52fx43fr0000gn/T/ipykernel_70379/2163845701.py:12: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  agent.data_schema = IncomeStatement_2024_25.schema()


In [6]:
agent.data_schema

{'additionalProperties': False,
 'properties': {'gross_tuition_revenue': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Gross tuition revenue before financial aid for 2024. Look for labels like 'Gross Tuition and Fees'. Exclude net tuition or post-discounted amounts."},
  'financial_aid': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "Institutional financial aid or tuition discounts for 2024. Labeled 'Scholarships', 'Grants', or 'Financial Aid'. Do not include federal aid."},
  'net_tuition_revenue': {'anyOf': [{'type': 'integer'}, {'type': 'null'}],
   'description': "Tuition revenue net of financial aid for 2024. Must be explicitly labeled as 'Net Tuition Revenue' or equivalent. Do not compute manually."},
  'federal_grants_contracts': {'anyOf': [{'type': 'integer'},
    {'type': 'null'}],
   'description': "Federal operating grants and contracts for 2024. Look for 'Federal Grants & Contracts' in operating revenue section. Exclude cap

The following two cell blocks extract all schools' info into one excel file per school.

In [7]:
def process_school(school_name, school_dir):
    combined   = {}
    first_keys = None

    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=[f"{FISCAL_YEAR - 1}-{str(FISCAL_YEAR)[-2:]}"])
        df.index.name = "Metric"
        outfile = os.path.join(OUTPUT_ROOT, f"{school_name}.xlsx")
        df.to_excel(outfile)
        print(f"Saved output to {outfile}")
    else:
        print(f"No PDF data found for {school_name}")

In [ ]:
# Loop over schools
for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue
    print(f"Processing school: {school}")
    process_school(school, school_dir)

print("Extraction complete.")

Processing school: BRADLEY_UNIVERSITY
Extracting data from Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf


Extracting files: 100%|██████████| 1/1 [00:48<00:00, 48.44s/it]


Extracting data from Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf


Extracting files: 100%|██████████| 1/1 [00:15<00:00, 15.46s/it]


Saved output to output_incomestatement_final/BRADLEY_UNIVERSITY.xlsx
Processing school: CORNELL_UNIVERSITY
Extracting data from 2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__788_KB_.pdf


...

Extracting files: 100%|██████████| 1/1 [00:10<00:00, 10.54s/it]


Extracting data from Audited_Financials_and_Operating_Data_for_the_year_ended_06_30_2024_Document2__174_KB_.pdf


Extracting files: 100%|██████████| 1/1 [00:11<00:00, 11.06s/it]

Saved output to output_incomestatement_final/ST_LOUIS_UNIVERSITY_US.xlsx
Extraction complete.


Extracting files: 100%|██████████| 1/1 [00:10<00:00, 10.34s/it]


The following cell block extracts all the schools' info into one excel sheet but in different tabs.

In [9]:
OUTPUT_FILE = os.path.join(OUTPUT_ROOT, "all_schools.xlsx")

writer = pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl")

for school in sorted(os.listdir(PDF_ROOT)):
    school_dir = os.path.join(PDF_ROOT, school)
    if not os.path.isdir(school_dir):
        continue

    combined   = {}
    first_keys = None
    for fname in sorted(os.listdir(school_dir)):
        if not fname.lower().endswith(".pdf"):
            continue
        path = os.path.join(school_dir, fname)
        print(f"Extracting data from {school}/{fname}")
        try:
            run  = agent.extract(path)
            data = run.data or {}
            if first_keys is None:
                first_keys = list(data.keys())
                combined  = {k: None for k in first_keys}
            for k, v in data.items():
                if v not in (None, "", []):
                    combined[k] = v
        except Exception as err:
            print(f"Skipped {fname}: {err}")

    if first_keys:
        df = pd.DataFrame.from_dict(combined, orient="index", columns=[f"{FISCAL_YEAR - 1}-{str(FISCAL_YEAR)[-2:]}"])
        df.index.name = "Metric"
        sheet_name = school[:31]
        df.to_excel(writer, sheet_name=sheet_name)
    else:
        print(f"No data for {school}.")

writer.close()
print(f"All schools written to {OUTPUT_FILE}")

Extracting data from BRADLEY_UNIVERSITY/Annual_Financial_Information_and_Operating_Data__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__227_KB_.pdf
Extracting data from BRADLEY_UNIVERSITY/Audited_Financial_Statements_or_ACFR__Rule_15c2-12__for_FY24_for_the_year_ended_05_31_2024__541_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/2024_Audited_Financial_Statements_for_the_year_ended_06_30_2024__788_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/2024_Operating_Data_for_the_year_ended_06_30_2024__109_KB_.pdf
Extracting data from CORNELL_UNIVERSITY/Incorporate_OS_by_Reference_as_of_04_25_2024__2.4_MB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Annual_Report_-_Corrected_for_the_year_ended_05_31_2024__130_KB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Annual_Report_for_the_year_ended_05_31_2024__129_KB_.pdf
Extracting data from CULINARY_INSTITUTE_OF_AMERICA_THE/2024_Audited_Financial_Statements_for_the_year_ended_05_31_2024__277_KB_.pdf
Extra

In [ ]:
#Combine all the tabs into one sheet if wanted
file_path   = "output_incomestatement_final/all_schools.xlsx"  #Change this if need be
output_path = "output_incomestatement_final/all_schools_combined.xlsx"  #Change this if need be
raw = pd.read_excel(file_path, sheet_name=None, index_col=0)

school_series = {
    school: df.iloc[:, 0]                      # first (only) value column
    for school, df in raw.items()
}

df_comb = pd.DataFrame(school_series).T
df_comb.index.name = "School"                 
df_comb.insert(0, "Year", f"{FISCAL_YEAR - 1}–{FISCAL_YEAR}")

with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    df_comb.to_excel(writer, sheet_name="Combined")

print("Saved:", output_path)

Saved: output_incomestatement_final/all_schools_combined.xlsx
